In [4]:
import os
from nltk.tokenize import RegexpTokenizer
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

# Define a simple configuration class
class Config:
    RNN_TYPE = 'LSTM'
    TEXT_WORDS_NUM = 100  # Adjust this based on your needs

cfg = Config()

class RNN_ENCODER(nn.Module):
    def _init_(self,
                 ntoken,
                 ninput=300,
                 drop_prob=0.5,
                 nhidden=128,
                 nlayers=1,
                 bidirectional=True):
        super(RNN_ENCODER, self)._init_()
        self.n_steps = cfg.TEXT_WORDS_NUM
        self.ntoken = ntoken
        self.ninput = ninput
        self.drop_prob = drop_prob
        self.nlayers = nlayers
        self.bidirectional = bidirectional
        self.rnn_type = cfg.RNN_TYPE
        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        self.nhidden = nhidden // self.num_directions

        self.define_module()
        self.init_weights()

    def define_module(self):
        self.encoder = nn.Embedding(self.ntoken, self.ninput)
        self.drop = nn.Dropout(self.drop_prob)
        if self.rnn_type == 'LSTM':
            self.rnn = nn.LSTM(self.ninput,
                               self.nhidden,
                               self.nlayers,
                               batch_first=True,
                               dropout=self.drop_prob,
                               bidirectional=self.bidirectional)
        elif self.rnn_type == 'GRU':
            self.rnn = nn.GRU(self.ninput,
                              self.nhidden,
                              self.nlayers,
                              batch_first=True,
                              dropout=self.drop_prob,
                              bidirectional=self.bidirectional)
        else:
            raise NotImplementedError

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(
                weight.new(self.nlayers * self.num_directions, bsz,
                           self.nhidden).zero_()),
                    Variable(
                        weight.new(self.nlayers * self.num_directions, bsz,
                                   self.nhidden).zero_()))
        else:
            return Variable(
                weight.new(self.nlayers * self.num_directions, bsz,
                           self.nhidden).zero_())

    def forward(self, captions, cap_lens, hidden, mask=None):
        emb = self.drop(self.encoder(captions))
        cap_lens = cap_lens.data.tolist()
        emb = pack_padded_sequence(emb, cap_lens, batch_first=True)
        output, hidden = self.rnn(emb, hidden)
        output = pad_packed_sequence(output, batch_first=True)[0]
        words_emb = output.transpose(1, 2)
        if self.rnn_type == 'LSTM':
            sent_emb = hidden[0].transpose(0, 1).contiguous()
        else:
            sent_emb = hidden.transpose(0, 1).contiguous()
        sent_emb = sent_emb.view(-1, self.nhidden * self.num_directions)
        return words_emb, sent_emb


class TextFileLoader:
    def _init_(self, folder_path):
        self.folder_path = folder_path
        self.vocabulary = None
        self.max_seq_length = 30  # Adjust this based on your dataset
        self.rnn_encoder = RNN_ENCODER(ntoken=10000, ninput=300)  # You need to adjust ntoken based on your vocabulary size

    def load_text_files(self):
        if os.path.exists(self.folder_path) and os.path.isdir(self.folder_path):
            text_files = [file for file in os.listdir(self.folder_path) if file.endswith(".txt")]
            all_captions = []
            for file_name in text_files:
                file_path = os.path.join(self.folder_path, file_name)
                with open(file_path, "r", encoding="utf-8") as file:
                    file_content = file.read()
                tokens = self.tokenize(file_content)
                all_captions.append(tokens)

            self.build_vocabulary(all_captions)

            for tokens in all_captions:
                indices = self.convert_to_indices(tokens)
                embeddings = self.get_embeddings(indices)
                print("Tokens:", tokens)
                print("Embeddings:", embeddings)
                print("-" * 50)

        else:
            print(f"The folder path '{self.folder_path}' is not valid.")

    def tokenize(self, input_text):
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(input_text.lower())
        return tokens

    def build_vocabulary(self, all_captions):
        all_words = [word for caption in all_captions for word in caption]
        unique_words = list(set(all_words))
        self.vocabulary = {word: idx + 1 for idx, word in enumerate(unique_words)}
        self.vocabulary['<PAD>'] = 0
        self.reverse_vocabulary = {idx: word for word, idx in self.vocabulary.items()}

    def convert_to_indices(self, tokens):
        indices = [self.vocabulary.get(word, self.vocabulary['<PAD>']) for word in tokens]
        if len(indices) < self.max_seq_length:
            indices.extend([self.vocabulary['<PAD>']] * (self.max_seq_length - len(indices)))
        else:
            indices = indices[:self.max_seq_length]
        return indices

    def get_embeddings(self, indices):
        # Convert indices to PyTorch tensor
        indices_tensor = torch.tensor(indices).unsqueeze(0)  # Add batch dimension

        # Initialize hidden state
        hidden = self.rnn_encoder.init_hidden(1)

        # Forward pass through the RNN_ENCODER
        words_emb, sent_emb = self.rnn_encoder(indices_tensor, torch.tensor([len(indices)]), hidden)

        return sent_emb

# Example usage:
folder_path = "/media/osama/26D25BC6D25B993F1/Seven Semester/Fyp/LAGAN-main/data/sketches/sketches/text"
loader = TextFileLoader(folder_path)
loader.load_text_files()

TypeError: TextFileLoader() takes no arguments

In [1]:
from nltk.tokenize import RegexpTokenizer
# Your input text
input_text = "This is an example sentence."

# Create a tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Tokenize the input text
tokens = tokenizer.tokenize(input_text.lower())  # Convert to lowercase for consistency

# Print the tokens
print(tokens)


['this', 'is', 'an', 'example', 'sentence']


In [2]:
import os
from nltk.tokenize import RegexpTokenizer
from PIL import Image
import torch
from torchvision import transforms

class TextImageLoader:
    def __init__(self, text_folder, image_folder):
        self.text_loader = TextFileLoader(text_folder)
        self.image_folder = image_folder
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Adjust the size as needed
            transforms.ToTensor(),
            # Add more transformations if necessary
        ])
        self.batch_size = 32  # Set your desired batch size
        self.current_batch = 0

    def load_data_batch(self):
        text_data = self.text_loader.load_text_files()
        image_data = self.load_image_data()

        # Ensure both text and image data have the same length
        min_length = min(len(text_data), len(image_data))
        text_data = text_data[:min_length]
        image_data = image_data[:min_length]

        # Organize data into batches
        for i in range(0, min_length, self.batch_size):
            text_batch = text_data[i:i+self.batch_size]
            image_batch = image_data[i:i+self.batch_size]

            # Process the batches (e.g., convert text to indices, load images)
            processed_text_batch = self.process_text_batch(text_batch)
            processed_image_batch = self.process_image_batch(image_batch)

            yield processed_text_batch, processed_image_batch

            self.current_batch += 1

    def load_image_data(self):
        # Implement logic to load image data (you may use PyTorch's DataLoader)
        # Assuming image_data is a list where each element is the file path of an image
        image_data = [...]  # Replace with your actual implementation
        return image_data

    def process_text_batch(self, text_batch):
        # Implement logic to convert text to indices for a batch
        # You can use the TextFileLoader's convert_to_indices method
        indices_batch = [self.text_loader.convert_to_indices(tokens) for tokens in text_batch]
        return torch.tensor(indices_batch)

    def process_image_batch(self, image_batch):
        # Implement logic to load and preprocess images for a batch
        # You can use PyTorch's DataLoader for image loading
        images = [Image.open(image_path).convert('RGB') for image_path in image_batch]
        images = [self.transform(image) for image in images]
        return torch.stack(images)

# Example usage:
text_folder_path = "/media/osama/26D25BC6D25B993F1/Seven Semester/Fyp/LAGAN-main/data/sketches/sketches/text"
image_folder_path = "/media/osama/26D25BC6D25B993F1/Seven Semester/Fyp/LAGAN-main/data/sketches/sketches/images"  # Replace with the actual path
loader = TextImageLoader(text_folder_path, image_folder_path)

for text_batch, image_batch in loader.load_data_batch():
    # Your training logic for each batch
    print(f"Processing Batch {loader.current_batch}")
    print("Text Batch:", text_batch)
    print("Image Batch Shape:", image_batch.shape)
    print("=" * 50)


ModuleNotFoundError: No module named 'torchvision'

In [3]:
import os
from nltk.tokenize import RegexpTokenizer
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

# Define a simple configuration class
class Config:
    RNN_TYPE = 'LSTM'
    TEXT_WORDS_NUM = 50  # Adjust this based on your needs

cfg = Config()

class RNN_ENCODER(nn.Module):
    def __init__(self,
                 ntoken,
                 ninput=30,
                 drop_prob=0.5,
                 nhidden=128,
                 nlayers=1,
                 bidirectional=True):
        super(RNN_ENCODER, self).__init__()
        self.n_steps = cfg.TEXT_WORDS_NUM
        self.ntoken = ntoken
        self.ninput = ninput
        self.drop_prob = drop_prob
        self.nlayers = nlayers
        self.bidirectional = bidirectional
        self.rnn_type = cfg.RNN_TYPE
        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        self.nhidden = nhidden // self.num_directions

        self.define_module()
        self.init_weights()

    def define_module(self):
        self.encoder = nn.Embedding(self.ntoken, self.ninput)
        self.drop = nn.Dropout(self.drop_prob)
        if self.rnn_type == 'LSTM':
            self.rnn = nn.LSTM(self.ninput,
                               self.nhidden,
                               self.nlayers,
                               batch_first=True,
                               dropout=self.drop_prob,
                               bidirectional=self.bidirectional)
        elif self.rnn_type == 'GRU':
            self.rnn = nn.GRU(self.ninput,
                              self.nhidden,
                              self.nlayers,
                              batch_first=True,
                              dropout=self.drop_prob,
                              bidirectional=self.bidirectional)
        else:
            raise NotImplementedError

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(
                weight.new(self.nlayers * self.num_directions, bsz,
                           self.nhidden).zero_()),
                    Variable(
                        weight.new(self.nlayers * self.num_directions, bsz,
                                   self.nhidden).zero_()))
        else:
            return Variable(
                weight.new(self.nlayers * self.num_directions, bsz,
                           self.nhidden).zero_())

    def forward(self, captions, cap_lens, hidden, mask=None):
        emb = self.drop(self.encoder(captions))
        cap_lens = cap_lens.data.tolist()
        emb = pack_padded_sequence(emb, cap_lens, batch_first=True)
        output, hidden = self.rnn(emb, hidden)
        output = pad_packed_sequence(output, batch_first=True)[0]
        words_emb = output.transpose(1, 2)
        if self.rnn_type == 'LSTM':
            sent_emb = hidden[0].transpose(0, 1).contiguous()
        else:
            sent_emb = hidden.transpose(0, 1).contiguous()
        sent_emb = sent_emb.view(-1, self.nhidden * self.num_directions)
        return words_emb, sent_emb


class TextFileLoader:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.vocabulary = None
        self.max_seq_length = 30  # Adjust this based on your dataset
        self.rnn_encoder = RNN_ENCODER(ntoken=10000, ninput=300)  # You need to adjust ntoken based on your vocabulary size

    def load_text_files(self):
        if os.path.exists(self.folder_path) and os.path.isdir(self.folder_path):
            text_files = [file for file in os.listdir(self.folder_path) if file.endswith(".txt")]
            all_captions = []
            for file_name in text_files:
                file_path = os.path.join(self.folder_path, file_name)
                with open(file_path, "r", encoding="utf-8") as file:
                    file_content = file.read()
                tokens = self.tokenize(file_content)
                all_captions.append(tokens)

            self.build_vocabulary(all_captions)
            

            for tokens in all_captions:
                indices = self.convert_to_indices(tokens)
                embeddings = self.get_embeddings(indices)
                print("Tokens:", tokens)
                print("Embeddings:", embeddings)
                print("-" * 50)

        else:
            print(f"The folder path '{self.folder_path}' is not valid.")

    def tokenize(self, input_text):
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(input_text.lower())
        return tokens

    def build_vocabulary(self, all_captions):
        all_words = [word for caption in all_captions for word in caption]
        unique_words = list(set(all_words))
        self.vocabulary = {word: idx + 1 for idx, word in enumerate(unique_words)}
        self.vocabulary['<PAD>'] = 0
        self.reverse_vocabulary = {idx: word for word, idx in self.vocabulary.items()}

    def convert_to_indices(self, tokens):
        indices = [self.vocabulary.get(word, self.vocabulary['<PAD>']) for word in tokens]
        if len(indices) < self.max_seq_length:
            indices.extend([self.vocabulary['<PAD>']] * (self.max_seq_length - len(indices)))
        else:
            indices = indices[:self.max_seq_length]
        return indices

    def get_embeddings(self, indices):
        # Convert indices to PyTorch tensor
        indices_tensor = torch.tensor(indices).unsqueeze(0)  # Add batch dimension

        # Initialize hidden state
        hidden = self.rnn_encoder.init_hidden(1)

        # Forward pass through the RNN_ENCODER
        words_emb, sent_emb = self.rnn_encoder(indices_tensor, torch.tensor([len(indices)]), hidden)

        return sent_emb

# Example usage:
folder_path = "/media/osama/26D25BC6D25B993F1/Seven Semester/Fyp/LAGAN-main/data/sketches/sketches/text"
loader = TextFileLoader(folder_path)
loader.load_text_files()


/home/osama/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Tokens: ['this', 'woman', 'has', 'a', 'rectangular', 'face', 'with', 'long', 'hair', 'she', 'has', 'a', 'pair', 'of', 'big', 'wide', 'eyes', 'with', 'dense', 'thin', 'and', 'flat', 'eyebrows', 'her', 'mouth', 'is', 'thick', 'and', 'wide', 'with', 'a', 'big', 'long', 'nose', 'and', 'her', 'ears', 'are', 'normal', 'she', 'hasn', 't', 'glasses', 'and', 'hasn', 't', 'beard']
Embeddings: tensor([[-9.7837e-02, -6.5076e-02, -8.6246e-02, -6.1002e-02,  2.2009e-02,
         -1.2662e-01,  3.0422e-02, -7.6951e-02,  3.1361e-03,  1.2886e-02,
          1.1706e-01, -8.8470e-02, -5.5904e-02, -8.2408e-02, -4.6922e-02,
         -7.5258e-02, -1.0929e-02, -5.2881e-02,  3.8481e-02,  1.3263e-02,
         -1.5592e-02,  4.8420e-03,  7.0873e-04,  3.8115e-02, -1.8950e-02,
         -5.1950e-02, -2.3912e-02, -5.1969e-02,  3.5623e-02,  4.2895e-02,
         -4.4600e-02, -8.6356e-02,  2.9311e-02, -6.2071e-02,  4.8905e-02,
          2.1449e-02, -5.7877e-02,  6.3482e-02, -9.9581e-03,  2.6566e-02,
         -8.8540e-02, 

In [4]:
import os
from nltk.tokenize import RegexpTokenizer

class TextFileLoader:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.vocabulary = None
        self.max_seq_length = 30  # Adjust this based on your dataset

    def load_text_files(self):
        if os.path.exists(self.folder_path) and os.path.isdir(self.folder_path):
            # Get a list of all text files in the folder
            text_files = [file for file in os.listdir(self.folder_path) if file.endswith(".txt")]

            # Build vocabulary and convert text to indices
            all_captions = []
            for file_name in text_files:
                file_path = os.path.join(self.folder_path, file_name)

                # Open the file and read its content
                with open(file_path, "r", encoding="utf-8") as file:
                    file_content = file.read()

                # Tokenize the content
                tokens = self.tokenize(file_content)
                
                # Build a vocabulary based on all captions
                all_captions.append(tokens)

            # Build the vocabulary
            self.build_vocabulary(all_captions)

            # Convert text to indices and print the result
            for tokens in all_captions:
                indices = self.convert_to_indices(tokens)
                print("Tokens:", tokens)
                print("Indices:", indices)
                print("-" * 50)

        else:
            print(f"The folder path '{self.folder_path}' is not valid.")

    def tokenize(self, input_text):
        # Create a tokenizer
        tokenizer = RegexpTokenizer(r'\w+')

        # Tokenize the input text
        tokens = tokenizer.tokenize(input_text.lower())  # Convert to lowercase for consistency
        return tokens

    def build_vocabulary(self, all_captions):
        # Flatten the list of all captions
        all_words = [word for caption in all_captions for word in caption]

        # Get unique words
        unique_words = list(set(all_words))

        # Create vocabulary and reverse vocabulary
        self.vocabulary = {word: idx + 1 for idx, word in enumerate(unique_words)}
        self.vocabulary['<PAD>'] = 0  # Add padding token
        self.reverse_vocabulary = {idx: word for word, idx in self.vocabulary.items()}

    def convert_to_indices(self, tokens):
        # Convert tokens to indices using the vocabulary
        indices = [self.vocabulary.get(word, self.vocabulary['<PAD>']) for word in tokens]

        # Perform padding or truncation to ensure a fixed sequence length
        if len(indices) < self.max_seq_length:
            indices.extend([self.vocabulary['<PAD>']] * (self.max_seq_length - len(indices)))
        else:
            indices = indices[:self.max_seq_length]

        return indices

# Example usage:
folder_path = "/media/osama/26D25BC6D25B993F1/Seven Semester/Fyp/LAGAN-main/data/sketches/sketches/text"
loader = TextFileLoader(folder_path)
loader.load_text_files()


Tokens: ['this', 'woman', 'has', 'a', 'rectangular', 'face', 'with', 'long', 'hair', 'she', 'has', 'a', 'pair', 'of', 'big', 'wide', 'eyes', 'with', 'dense', 'thin', 'and', 'flat', 'eyebrows', 'her', 'mouth', 'is', 'thick', 'and', 'wide', 'with', 'a', 'big', 'long', 'nose', 'and', 'her', 'ears', 'are', 'normal', 'she', 'hasn', 't', 'glasses', 'and', 'hasn', 't', 'beard']
Indices: [37, 19, 28, 47, 13, 31, 27, 39, 34, 15, 28, 47, 3, 52, 16, 22, 14, 27, 17, 42, 20, 7, 38, 51, 10, 49, 9, 20, 22, 27]
--------------------------------------------------
Tokens: ['this', 'woman', 'has', 'an', 'inverted', 'triangle', 'face', 'with', 'short', 'hair', 'he', 'has', 'a', 'pair', 'of', 'big', 'wide', 'eyes', 'with', 'dense', 'thin', 'and', 'arched', 'eyebrows', 'her', 'mouth', 'is', 'thin', 'and', 'wide', 'with', 'a', 'medium', 'normal', 'nose', 'and', 'her', 'ears', 'are', 'small', 'she', 'hasn', 't', 'glasses', 'and', 'hasn', 't', 'beard']
Indices: [37, 19, 28, 40, 25, 6, 31, 27, 32, 34, 21, 28, 47

In [1]:
import os
from nltk.tokenize import RegexpTokenizer

class TextFileLoader:
    def __init__(self, folder_path):
        self.folder_path = folder_path

    def load_text_files(self):
        if os.path.exists(self.folder_path) and os.path.isdir(self.folder_path):
            # Get a list of all text files in the folder
            text_files = [file for file in os.listdir(self.folder_path) if file.endswith(".txt")]

            # Iterate through each text file and load its content
            for file_name in text_files:
                file_path = os.path.join(self.folder_path, file_name)

                # Open the file and read its content
                with open(file_path, "r", encoding="utf-8") as file:
                    file_content = file.read()

                # Process the file content as needed (you can customize this part)
                print(f"File: {file_name}")
                print("Content:")
                print(file_content)
                
                # Tokenize the content
                tokens = self.tokenize(file_content)
                
                # Print the tokens
                print("Tokens:")
                print(tokens)
                print("-" * 50)

        else:
            print(f"The folder path '{self.folder_path}' is not valid.")

    def tokenize(self, input_text):
        # Create a tokenizer
        tokenizer = RegexpTokenizer(r'\w+')

        # Tokenize the input text
        tokens = tokenizer.tokenize(input_text.lower())  # Convert to lowercase for consistency
        return tokens

# Example usage:
folder_path = "/media/osama/26D25BC6D25B993F1/Seven Semester/Fyp/LAGAN-main/data/sketches/sketches/text"
loader = TextFileLoader(folder_path)
loader.load_text_files()


File: 00360.txt
Content:
this woman has a rectangular face with long hair.
she has a pair of big wide eyes,with dense thin and flat eyebrows.
her mouth is thick and wide,with a big long nose and her ears are normal.
she hasn't glasses and hasn't beard.

Tokens:
['this', 'woman', 'has', 'a', 'rectangular', 'face', 'with', 'long', 'hair', 'she', 'has', 'a', 'pair', 'of', 'big', 'wide', 'eyes', 'with', 'dense', 'thin', 'and', 'flat', 'eyebrows', 'her', 'mouth', 'is', 'thick', 'and', 'wide', 'with', 'a', 'big', 'long', 'nose', 'and', 'her', 'ears', 'are', 'normal', 'she', 'hasn', 't', 'glasses', 'and', 'hasn', 't', 'beard']
--------------------------------------------------
File: 00720.txt
Content:
this woman has an inverted triangle face with short hair.
he has a pair of big wide eyes,with dense thin and arched eyebrows.
her mouth is thin and wide,with a medium normal nose and her ears are small.
she hasn’t glasses and hasn’t beard.

Tokens:
['this', 'woman', 'has', 'an', 'inverted', 'tri